In [59]:
from algorithms.persistance import load_from_json

import os
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [60]:
def convert_to_df(path: str):
    results = load_from_json(path)
    config_list = []
    performance_list = []
    for result in results:
        config_list.append(result['config'])
        performance_list.append(result['performance'])
    config = pd.DataFrame(config_list)
    performance = pd.DataFrame(performance_list)
    result_list = pd.concat([performance, config], axis=1)
    return result_list

In [61]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# show exerpt of json file as df

In [62]:

de = 'merged'
path = f'../persistent_data/{de}.json'
result_list = convert_to_df(path)
result_list

,true_positives,false_positives,true_negatives,false_negatives,alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,...,precision_with_syscalls,scenario,ngram,batch_size,embedding_size,return_value,thread_change_flag,time_delta,detection_time,train_time
0,173,1,1229731,194416,172,174,0.988506,1,0,0.988506,...,0.994220,CVE-2017-7529/,2,1024,4,False,False,False,14.714413,NaN
1,0,0,1226104,194589,0,174,0.000000,0,0,0.000000,...,0.000000,CVE-2017-7529/,6,1024,4,False,False,False,15.412062,NaN
2,0,14,2836396,285051,0,100,0.000000,14,0,0.000000,...,0.000000,CVE-2014-0160/,2,1024,4,False,False,False,33.687482,NaN
3,1,0,2833010,285050,1,100,0.010000,0,0,0.010000,...,1.000000,CVE-2014-0160/,6,1024,4,False,False,False,34.964247,NaN
4,23,17,2668699,2095470,18,98,0.183673,15,2,0.183673,...,0.514286,Bruteforce_CWE-307/,6,1024,4,False,False,False,41.704447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2693,25,86968068,9934032,99,99,1.000000,25,0,1.000000,...,0.798387,EPS_CWE-434/,6,1024,4,True,True,True,1208.838838,782.406823
679,2073,2,86971375,9934652,99,99,1.000000,2,0,1.000000,...,0.980198,EPS_CWE-434/,2,1024,4,True,True,True,1168.478725,774.014723
680,117,71,173496604,19582702,29,98,0.295918,70,1,0.295918,...,0.290000,ZipSlip/,2,1024,6,False,False,False,2161.861879,1578.773005
681,1234,8,86964801,9935491,99,99,1.000000,8,0,1.000000,...,0.925234,EPS_CWE-434/,10,1024,4,False,True,True,1188.505251,785.770628


# connect to db and create table

In [63]:
#conn = sql.connect('evaluation.db')
conn = sql.connect('Whole.db')
try:
    result_list.to_sql(de, conn)
except ValueError:
    print('Table already exists')

In [64]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])

merged


# show all different configs DR > 0.5

In [65]:
#conn = sql.connect('evaluation.db')
#conn = sql.connect('zwischenerg.db')
result = pd.read_sql(f'SELECT \
                             ngram, \
                             embedding_size as embed, \
                             return_value as rv, \
                             thread_change_flag as tcf, \
                             time_delta as td, \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             ROUND(AVG(detection_rate),2) as DR, \
                             COUNT(*) \
                     FROM  {de}\
                     GROUP BY ngram, \
                              embedding_size, \
                              rv, \
                              tcf, \
                              td \
                     ORDER BY  \
                              DR desc, \
                              fp_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,ngram,embed,rv,tcf,td,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,6,8,1,1,1,21.89,1.00,206,0.88,9
1,10,8,0,1,1,18.00,0.56,167,0.76,9
2,10,4,1,1,1,21.33,1.00,201,0.74,9
3,10,4,0,1,0,4.88,0.25,41,0.71,8
4,6,8,1,1,0,8.56,0.44,81,0.70,9
5,2,4,1,0,0,12.25,0.25,100,0.70,8
6,10,4,1,1,0,12.00,0.63,101,0.70,8
7,10,4,1,0,0,13.75,0.63,115,0.70,8
8,2,8,1,1,1,13.60,0.40,140,0.70,10
9,6,4,1,1,1,10.33,0.56,98,0.69,9


# show all scenarios for best result

In [78]:
# 6 	8 	1 	1 	1 
# 10 	4 	0 	1 	0
result = pd.read_sql(f'SELECT \
                             scenario, \
                             ngram, \
                             embedding_size as embed, \
                             return_value as rv, \
                             thread_change_flag as tcf, \
                             time_delta as td, \
                             consecutive_false_positives_normal as fp_norm, \
                             consecutive_false_positives_exploits as fp_ex, \
                             consecutive_false_positives_normal + consecutive_false_positives_exploits as fp_sum, \
                             detection_rate as DR \
                     FROM {de} \
                     WHERE ngram = 6 AND embed=8 AND rv=1 AND td=1 AND tcf=1', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,ngram,embed,rv,tcf,td,fp_norm,fp_ex,fp_sum,DR
0,Bruteforce_CWE-307/,6,8,1,1,1,25,4,29,0.938776
1,CVE-2012-2122/,6,8,1,1,1,18,0,18,0.987097
2,CVE-2017-7529/,6,8,1,1,1,0,0,0,0.988506
3,CVE-2014-0160/,6,8,1,1,1,13,0,13,0.030000
4,CVE-2018-3760/,6,8,1,1,1,4,0,4,1.000000
5,CVE-2019-5418/,6,8,1,1,1,7,0,7,1.000000
6,PHP_CWE-434/,6,8,1,1,1,22,2,24,1.000000
7,SQL_Injection_CWE-89/,6,8,1,1,1,79,3,82,1.000000
8,EPS_CWE-434/,6,8,1,1,1,29,0,29,1.000000


In [79]:
# 2 	8 	1 	1 	1 
result = pd.read_sql(f'SELECT \
                             scenario, \
                             ngram, \
                             embedding_size as embed, \
                             return_value as rv, \
                             thread_change_flag as tcf, \
                             time_delta as td, \
                             consecutive_false_positives_normal as fp_norm, \
                             consecutive_false_positives_exploits as fp_ex, \
                             consecutive_false_positives_normal + consecutive_false_positives_exploits as fp_sum, \
                             detection_rate as DR \
                     FROM {de} \
                     WHERE ngram = 2 AND embed=8 AND rv=1 AND td=1 AND tcf=1', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,ngram,embed,rv,tcf,td,fp_norm,fp_ex,fp_sum,DR
0,Bruteforce_CWE-307/,2,8,1,1,1,54,3,57,0.724490
1,CVE-2017-7529/,2,8,1,1,1,1,0,1,0.988506
2,CVE-2012-2122/,2,8,1,1,1,1,0,1,0.000000
3,CVE-2014-0160/,2,8,1,1,1,2,0,2,0.010000
4,CVE-2019-5418/,2,8,1,1,1,1,0,1,1.000000
5,CVE-2018-3760/,2,8,1,1,1,0,0,0,1.000000
6,PHP_CWE-434/,2,8,1,1,1,0,0,0,1.000000
7,SQL_Injection_CWE-89/,2,8,1,1,1,39,1,40,1.000000
8,EPS_CWE-434/,2,8,1,1,1,21,0,21,1.000000
9,ZipSlip/,2,8,1,1,1,17,0,17,0.275510


# comparison with and without time_delta 

In [81]:
# show results for best config
# reverse fp und dr
result = pd.read_sql(f'SELECT scenario, time_delta as td,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR \
                      FROM {de} \
                      GROUP BY scenario, td \
                      ORDER BY scenario, DR desc, fp_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,td,fp_norm,fp_ex,fp_sum,DR
0,Bruteforce_CWE-307/,1,27.53,1.75,1054,0.324546
1,Bruteforce_CWE-307/,0,20.39,1.39,893,0.251867
2,CVE-2012-2122/,1,9.75,0.44,367,0.039606
3,CVE-2012-2122/,0,11.80,0.49,504,0.018725
4,CVE-2014-0160/,1,6.31,0.17,233,0.011111
5,CVE-2014-0160/,0,4.59,0.12,193,0.005854
6,CVE-2017-7529/,1,1.50,0.00,54,0.906290
7,CVE-2017-7529/,0,0.83,0.00,34,0.868517
8,CVE-2018-3760/,1,9.72,0.42,365,1.000000
9,CVE-2018-3760/,0,9.41,0.63,412,1.000000


# comparison with and without return_value

In [83]:
result = pd.read_sql(f'SELECT scenario, return_value as rv,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR \
                      FROM {de} \
                      GROUP BY scenario, rv \
                      ORDER BY scenario, DR desc, fp_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,rv,fp_norm,fp_ex,fp_sum,DR
0,Bruteforce_CWE-307/,1,24.13,1.44,997,0.311094
1,Bruteforce_CWE-307/,0,23.32,1.68,950,0.259936
2,CVE-2012-2122/,1,13.24,0.58,525,0.039728
3,CVE-2012-2122/,0,8.51,0.36,346,0.017535
4,CVE-2014-0160/,1,7.34,0.24,288,0.011842
5,CVE-2014-0160/,0,3.49,0.05,138,0.004872
6,CVE-2017-7529/,1,0.92,0.00,35,0.910617
7,CVE-2017-7529/,0,1.36,0.00,53,0.862364
8,CVE-2018-3760/,1,9.18,0.26,359,1.000000
9,CVE-2018-3760/,0,9.92,0.79,418,1.000000


# comparison with and without thread_change_flag

In [94]:
result = pd.read_sql(f'SELECT scenario, thread_change_flag as tcf,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR, \
                             COUNT(*) \
                      FROM {de} \
                      GROUP BY scenario, tcf \
                      ORDER BY scenario, DR desc, fp_ex asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,tcf,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,Bruteforce_CWE-307/,1,27.86,1.81,1068,0.402494,36
1,Bruteforce_CWE-307/,0,20.10,1.34,879,0.183425,41
2,CVE-2012-2122/,1,4.83,0.08,177,0.042652,36
3,CVE-2012-2122/,0,16.12,0.80,694,0.016050,41
4,CVE-2014-0160/,1,4.50,0.17,168,0.008333,36
5,CVE-2014-0160/,0,6.17,0.12,258,0.008293,41
6,CVE-2017-7529/,1,1.50,0.00,54,0.933589,36
7,CVE-2017-7529/,0,0.83,0.00,34,0.844547,41
8,CVE-2018-3760/,1,7.78,0.44,296,1.000000,36
9,CVE-2018-3760/,0,11.12,0.61,481,1.000000,41


# comparison of all extra params for each scenario

In [87]:
result = pd.read_sql(f'SELECT scenario, time_delta as td, thread_change_flag as tcf, return_value as rv,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR, \
                             COUNT(*)\
                      FROM {de} \
                      GROUP BY scenario, td, tcf, rv \
                      ORDER BY scenario, DR desc, fp_norm asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,td,tcf,rv,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,Bruteforce_CWE-307/,1,1,1,33.11,2.00,316,0.501134,9
1,Bruteforce_CWE-307/,0,1,1,30.00,2.00,288,0.386621,9
2,Bruteforce_CWE-307/,0,1,0,19.44,1.33,187,0.365079,9
3,Bruteforce_CWE-307/,1,1,0,28.89,1.89,277,0.357143,9
4,Bruteforce_CWE-307/,1,0,1,23.33,0.89,218,0.278912,9
5,Bruteforce_CWE-307/,0,0,0,20.73,1.36,243,0.175325,11
6,Bruteforce_CWE-307/,1,0,0,24.78,2.22,243,0.160998,9
7,Bruteforce_CWE-307/,0,0,1,13.58,1.00,175,0.136054,12
8,CVE-2012-2122/,1,1,1,5.67,0.11,52,0.121864,9
9,CVE-2012-2122/,0,1,0,6.78,0.00,61,0.031541,9


In [95]:
result = pd.read_sql(f'SELECT \
                             scenario, \
                             ngram, \
                             embedding_size as embed, \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             ROUND(AVG(detection_rate),2) as DR, \
                             COUNT(*) \
                     FROM {de} \
                     GROUP BY scenario, \
                              ngram, \
                              embedding_size \
                     ORDER BY scenario, \
                              DR desc, \
                              fp_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,ngram,embed,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,Bruteforce_CWE-307/,10,4,31.25,1.88,265,0.59,8
1,Bruteforce_CWE-307/,10,8,25.14,1.86,189,0.45,7
2,Bruteforce_CWE-307/,2,6,44.50,2.63,377,0.37,8
3,Bruteforce_CWE-307/,2,4,48.38,2.25,405,0.36,8
4,Bruteforce_CWE-307/,2,8,30.13,1.38,252,0.31,8
5,Bruteforce_CWE-307/,6,8,9.38,1.13,84,0.22,8
6,Bruteforce_CWE-307/,6,6,11.50,1.25,102,0.20,8
7,Bruteforce_CWE-307/,6,4,14.25,2.00,130,0.18,8
8,Bruteforce_CWE-307/,10,6,11.25,0.75,96,0.10,8
9,Bruteforce_CWE-307/,10,10,7.00,0.50,15,0.06,2


In [96]:
result = pd.read_sql(f'SELECT scenario, \
                             ngram as n, \
                             embedding_size as embed, \
                             time_delta as td, \
                             thread_change_flag as tcf, \
                             return_value as rv, \
                             consecutive_false_positives_normal as fp_norm, \
                             consecutive_false_positives_exploits as fp_exp, \
                             detection_rate as dr \
                      FROM {de} \
                      WHERE dr = (SELECT MAX(detection_rate) from {de}) \
                      ORDER BY scenario, \
                               fp_norm asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,n,embed,td,tcf,rv,fp_norm,fp_exp,dr
0,CVE-2018-3760/,2,4,0,1,1,0,0,1.0
1,CVE-2018-3760/,2,6,0,1,0,0,0,1.0
2,CVE-2018-3760/,2,8,0,1,0,0,0,1.0
3,CVE-2018-3760/,10,8,0,1,0,0,0,1.0
4,CVE-2018-3760/,6,8,1,0,0,0,0,1.0
5,CVE-2018-3760/,2,8,1,1,0,0,0,1.0
6,CVE-2018-3760/,2,8,1,1,1,0,0,1.0
7,CVE-2018-3760/,10,4,0,1,0,1,0,1.0
8,CVE-2018-3760/,2,6,0,0,0,1,0,1.0
9,CVE-2018-3760/,6,6,0,1,0,1,0,1.0


In [58]:
result = pd.read_sql(f'SELECT * \
                     FROM {de}', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,index,true_positives,false_positives,true_negatives,false_negatives,alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,...,precision_with_syscalls,scenario,ngram,batch_size,embedding_size,return_value,thread_change_flag,time_delta,detection_time,train_time
0,0,1931,29,86968064,9934794,99,99,1.000000,29,0,...,0.773438,EPS_CWE-434/,6,1024,8,1,1,1,1227.200028,798.789983
1,1,0,0,86964809,9936725,0,99,0.000000,0,0,...,0.000000,EPS_CWE-434/,10,1024,8,1,1,1,1250.643337,808.848570
2,2,1804,21,86971356,9934921,99,99,1.000000,21,0,...,0.825000,EPS_CWE-434/,2,1024,8,1,1,1,1195.196509,767.587275
3,3,1424,9,86964800,9935301,99,99,1.000000,8,1,...,0.916667,EPS_CWE-434/,10,1024,8,0,1,1,1261.900041,829.679385
4,4,83,17,173496658,19582736,27,98,0.275510,17,0,...,0.613636,ZipSlip/,2,1024,8,1,1,1,2365.030365,1569.074479
5,5,893,31,86968062,9935832,99,99,1.000000,31,0,...,0.761538,EPS_CWE-434/,6,1024,8,0,1,1,1274.352357,820.931523
6,6,985,1,86971376,9935740,99,99,1.000000,1,0,...,0.990000,EPS_CWE-434/,2,1024,8,0,1,1,1174.837613,798.510102
7,7,1285,8,86964801,9935440,99,99,1.000000,8,0,...,0.925234,EPS_CWE-434/,10,1024,8,1,0,1,1263.653508,842.960162
8,8,908,7,86968086,9935817,99,99,1.000000,7,0,...,0.933962,EPS_CWE-434/,6,1024,8,1,0,1,1266.647309,813.869955
9,9,1394,51,86971326,9935331,99,99,1.000000,51,0,...,0.660000,EPS_CWE-434/,2,1024,8,1,0,1,1194.903163,809.549815
